# Aggregating counts and rates of hospital admissions and make a table

In [1]:
import pandas as pd
import numpy as np
import functools as ft

## 0- functions and read data

In [2]:
def data_process(data, data_hosped, infection):
    #replace SMs with 5 in data and data_hosped and NaN with 0 in data_hosped 
    data = data.replace({'SM': 5})
    data_hosped = data_hosped.replace({'SM': 5, np.nan:0})
    #conver counts into int
    data[infection] = data[infection].astype(str).replace('\.0', '', regex=True).astype(int)
    data_hosped[infection] = data_hosped[infection].astype(str).replace('\.0', '', regex=True).astype(int)

    #add hosp rate
    data_hosped[infection+'_rate'] = (data_hosped[infection]/data[infection])*1000
    data_hosped[infection+'_rate'] = np.round(data_hosped[infection+'_rate'], decimals=1)

    #replace rate with SN if infection count equals 5
    data_hosped.loc[data_hosped[infection] == 5, infection+'_rate'] = 'SN'

    #calculate total using sex_female and sex_male
    total = data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_male'].index[0])][infection] + data_hosped.iloc[int(data_hosped[data_hosped['variable']=='sex_female'].index[0])][infection]
    total = "{:,}".format(total)
    data_hosped['variable'] = data_hosped['variable'].str.replace('antibacterial_brit', 'total')
    
    #add comma to thousands of column count
    data_hosped[infection] = data_hosped[infection].apply(lambda x : "{:,}".format(x))
    #make new column with counts and rates
    data_hosped[infection+'_rate_count'] = data_hosped[infection].astype(str) + ' (' + data_hosped[infection+'_rate'].astype(str) + ')'
    
    #replace total (after making new column of counts and rates)
    data_hosped.at[int(data_hosped[data_hosped['variable']=='total'].index[0]),infection+'_rate_count'] = total

    #drop redundant columns and rename the needed column
    data_hosped = data_hosped.drop([infection, infection+'_rate'], axis=1)
    data_hosped = data_hosped.rename(columns={infection+'_rate_count': infection})
    #replace any row with no value (=0) with a dash 
    data_hosped[infection] = data_hosped[infection].replace({'0 (0.0)': '-'})
    #replace count and rate if rate contains SN
    data_hosped.loc[data_hosped[infection].str.contains('SN', regex=False), infection] = 'SN' 

    return data_hosped

In [3]:
summary_table_cases_lrti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_incdt_cases.csv')
summary_table_cases_lrti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_prevl_cases.csv')
summary_table_cases_urti_combined_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_incdt_cases.csv')
summary_table_cases_urti_combined_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_prevl_cases.csv')
summary_table_cases_uti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_incdt_cases.csv')
summary_table_cases_uti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_prevl_cases.csv')
summary_table_cases_sinusitis_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_incdt_cases.csv')
summary_table_cases_sinusitis_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_prevl_cases.csv')
summary_table_cases_otmedia_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_incdt_cases.csv')
summary_table_cases_otmedia_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_prevl_cases.csv')
summary_table_cases_ot_externa_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_incdt_cases.csv')
summary_table_cases_ot_externa_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_prevl_cases.csv')
summary_table_cases_urti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_incdt_cases.csv')
summary_table_cases_urti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_prevl_cases.csv')
summary_table_cases_cough_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_incdt_cases.csv')
summary_table_cases_cough_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_prevl_cases.csv')
summary_table_cases_cough_cold_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_incdt_cases.csv')
summary_table_cases_cough_cold_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_prevl_cases.csv')
summary_table_cases_throat_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_incdt_cases.csv')
summary_table_cases_throat_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_prevl_cases.csv')

In [4]:
summary_table_lrti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_incdt.csv')
summary_table_lrti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_lrti/summary_table_lrti_prevl.csv')
summary_table_urti_combined_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_incdt.csv')
summary_table_urti_combined_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti_combined/summary_table_urti_combined_prevl.csv')
summary_table_uti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_incdt.csv')
summary_table_uti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_uti/summary_table_uti_prevl.csv')
summary_table_sinusitis_incdt = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_incdt.csv')
summary_table_sinusitis_prevl = pd.read_csv(f'../output/hospitalisation_prediction_sinusitis/summary_table_sinusitis_prevl.csv')
summary_table_otmedia_incdt = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_incdt.csv')
summary_table_otmedia_prevl = pd.read_csv(f'../output/hospitalisation_prediction_otmedia/summary_table_otmedia_prevl.csv')
summary_table_ot_externa_incdt = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_incdt.csv')
summary_table_ot_externa_prevl = pd.read_csv(f'../output/hospitalisation_prediction_ot_externa/summary_table_ot_externa_prevl.csv')
summary_table_urti_incdt = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_incdt.csv')
summary_table_urti_prevl = pd.read_csv(f'../output/hospitalisation_prediction_urti/summary_table_urti_prevl.csv')
summary_table_cough_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_incdt.csv')
summary_table_cough_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough/summary_table_cough_prevl.csv')
summary_table_cough_cold_incdt = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_incdt.csv')
summary_table_cough_cold_prevl = pd.read_csv(f'../output/hospitalisation_prediction_cough_cold/summary_table_cough_cold_prevl.csv')
summary_table_throat_incdt = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_incdt.csv')
summary_table_throat_prevl = pd.read_csv(f'../output/hospitalisation_prediction_throat/summary_table_throat_prevl.csv')

In [5]:
#select relevant columns of each df
summary_table_lrti_incdt = summary_table_lrti_incdt[['variable', 'count']]
summary_table_lrti_prevl = summary_table_lrti_prevl[['variable', 'count']]
summary_table_urti_combined_incdt = summary_table_urti_combined_incdt[['variable', 'count']]
summary_table_urti_combined_prevl = summary_table_urti_combined_prevl[['variable', 'count']]
summary_table_uti_incdt = summary_table_uti_incdt[['variable', 'count']]
summary_table_uti_prevl = summary_table_uti_prevl[['variable', 'count']]
summary_table_sinusitis_incdt = summary_table_sinusitis_incdt[['variable', 'count']]
summary_table_sinusitis_prevl = summary_table_sinusitis_prevl[['variable', 'count']]
summary_table_otmedia_incdt = summary_table_otmedia_incdt[['variable', 'count']]
summary_table_otmedia_prevl = summary_table_otmedia_prevl[['variable', 'count']]
summary_table_ot_externa_incdt = summary_table_ot_externa_incdt[['variable', 'count']]
summary_table_ot_externa_prevl = summary_table_ot_externa_prevl[['variable', 'count']]
summary_table_urti_incdt = summary_table_urti_incdt[['variable', 'count']]
summary_table_urti_prevl = summary_table_urti_prevl[['variable', 'count']]
summary_table_cough_incdt = summary_table_cough_incdt[['variable', 'count']]
summary_table_cough_prevl = summary_table_cough_prevl[['variable', 'count']]
summary_table_cough_cold_incdt = summary_table_cough_cold_incdt[['variable', 'count']]
summary_table_cough_cold_prevl = summary_table_cough_cold_prevl[['variable', 'count']]
summary_table_throat_incdt = summary_table_throat_incdt[['variable', 'count']]
summary_table_throat_prevl = summary_table_throat_prevl[['variable', 'count']]

In [6]:
#merge baseline dfs
dfs_tab = [summary_table_lrti_incdt, summary_table_lrti_prevl, 
        summary_table_urti_combined_incdt, summary_table_urti_combined_prevl, 
        summary_table_uti_incdt, summary_table_uti_prevl, 
        summary_table_sinusitis_incdt, summary_table_sinusitis_prevl, 
        summary_table_otmedia_incdt, summary_table_otmedia_prevl, 
        summary_table_ot_externa_incdt, summary_table_ot_externa_prevl, 
        summary_table_urti_incdt, summary_table_urti_prevl, 
        summary_table_cough_incdt, summary_table_cough_prevl, 
        summary_table_cough_cold_incdt, summary_table_cough_cold_prevl, 
        summary_table_throat_incdt, summary_table_throat_prevl, 
        ]

#replace any column that contains ab_date with ab_binray
for i in dfs_tab:
    i.loc[i['variable'].str.contains('_ab_date'), 'variable'] = 'ab_binary'

#concat dfs and assign column names
df_concated = pd.concat([i.set_index('variable') for i in dfs_tab],axis=1, join='outer').reset_index()
df_concated.columns = ['variable', 
                        'lrti_incdt', 'lrti_prevl', 
                        'urti_combined_incdt', 'urti_combined_prevl',
                        'uti_incdt', 'uti_prevl',
                        'sinusitis_incdt', 'sinusitis_prevl',
                        'otmedia_incdt', 'otmedia_prevl',
                        'ot_externa_incdt', 'ot_externa_prevl',
                        'urti_incdt', 'urti_prevl',
                        'cough_incdt', 'cough_prevl',
                        'cough_cold_incdt', 'cough_cold_prevl',
                        'throat_incdt', 'throat_prevl'
                        ]

#custom sort baseline dfs
df_concated['variable'] = pd.Categorical(df_concated['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more',
                                                                   'sex_female', 'sex_male',
                                                                   'CCI_cat_high', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_very_high', 'CCI_cat_very_low',
                                                                   'flu_vaccine_no', 'flu_vaccine_yes',
                                                                   'bmi_cat_healthy_weight', 'bmi_cat_obese', 'bmi_cat_overweight', 'bmi_cat_underweight', 'bmi_cat_unknown',
                                                                   'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire',
                                                                   'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_unknown', 'imd_very_affluent', 'imd_very_unaffluent',
                                                                   'ethnicity_asian', 'ethnicity_other', 'ethnicity_unknown', 'ethnicity_white',
                                                                   'smoking_ex_smoker', 'smoking_never_smoked', 'smoking_smoker', 'smoking_unknown',
                                                                   'season_autumn', 'season_spring', 'season_summer', 'season_winter',
                                                                   'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic',
                                                                   'ab_binary', 'antibacterial_brit'])

df_final_tab = df_concated.sort_values("variable").reset_index(drop=True)
# df_final_tab

In [7]:
#select relevant columns of each df
summary_table_cases_lrti_incdt = summary_table_cases_lrti_incdt[['variable', 'count']]
summary_table_cases_lrti_prevl = summary_table_cases_lrti_prevl[['variable', 'count']]
summary_table_cases_urti_combined_incdt = summary_table_cases_urti_combined_incdt[['variable', 'count']]
summary_table_cases_urti_combined_prevl = summary_table_cases_urti_combined_prevl[['variable', 'count']]
summary_table_cases_uti_incdt = summary_table_cases_uti_incdt[['variable', 'count']]
summary_table_cases_uti_prevl = summary_table_cases_uti_prevl[['variable', 'count']]
summary_table_cases_sinusitis_incdt = summary_table_cases_sinusitis_incdt[['variable', 'count']]
summary_table_cases_sinusitis_prevl = summary_table_cases_sinusitis_prevl[['variable', 'count']]
summary_table_cases_otmedia_incdt = summary_table_cases_otmedia_incdt[['variable', 'count']]
summary_table_cases_otmedia_prevl = summary_table_cases_otmedia_prevl[['variable', 'count']]
summary_table_cases_ot_externa_incdt = summary_table_cases_ot_externa_incdt[['variable', 'count']]
summary_table_cases_ot_externa_prevl = summary_table_cases_ot_externa_prevl[['variable', 'count']]
summary_table_cases_urti_incdt = summary_table_cases_urti_incdt[['variable', 'count']]
summary_table_cases_urti_prevl = summary_table_cases_urti_prevl[['variable', 'count']]
summary_table_cases_cough_incdt = summary_table_cases_cough_incdt[['variable', 'count']]
summary_table_cases_cough_prevl = summary_table_cases_cough_prevl[['variable', 'count']]
summary_table_cases_cough_cold_incdt = summary_table_cases_cough_cold_incdt[['variable', 'count']]
summary_table_cases_cough_cold_prevl = summary_table_cases_cough_cold_prevl[['variable', 'count']]
summary_table_cases_throat_incdt = summary_table_cases_throat_incdt[['variable', 'count']]
summary_table_cases_throat_prevl = summary_table_cases_throat_prevl[['variable', 'count']]

In [8]:
# summary_table_cases_lrti_incdt.head()

In [9]:
# dfs_tab_cases = [summary_table_cases_lrti_incdt, summary_table_cases_lrti_prevl,  
#                 summary_table_cases_urti_combined_incdt, summary_table_cases_urti_combined_prevl, 
#                 summary_table_cases_uti_incdt, summary_table_cases_uti_prevl, 
#                 summary_table_cases_sinusitis_incdt, summary_table_cases_sinusitis_prevl, 
#                 summary_table_cases_otmedia_incdt, summary_table_cases_otmedia_prevl, 
#                 summary_table_cases_ot_externa_incdt, summary_table_cases_ot_externa_prevl, 
#                 summary_table_cases_urti_incdt, summary_table_cases_urti_prevl, 
#                 summary_table_cases_cough_incdt, summary_table_cases_cough_prevl, 
#                 summary_table_cases_cough_cold_incdt, summary_table_cases_cough_cold_prevl, 
#                 summary_table_cases_throat_incdt, summary_table_cases_throat_prevl, 
#                 ]

# for i in dfs_tab_cases:
#     print(str(i))
#      # for j in dfs_tab_cases:
#      #      if i['variable'].tolist() == j['variable'].tolist():
#      #           print('TRUE')
#      #      else:
#      #           print('FALSE' and i)# + j)

summary_table_cases_lrti_incdt

,variable,count
0,antibacterial_brit,19800.0
1,age_cat_15_24,2335.0
2,age_cat_25_34,2200.0
3,age_cat_35_44,2305.0
4,age_cat_45_54,2275.0
5,age_cat_55_64,2320.0
6,age_cat_65_74,2360.0
7,age_cat_75_more,2300.0
8,sex_female,8245.0
9,sex_male,7850.0


In [10]:
summary_table_cases_lrti_prevl

,variable,count
0,antibacterial_brit,2140.0
1,age_cat_15_24,275.0
2,age_cat_25_34,265.0
3,age_cat_35_44,265.0
4,age_cat_45_54,235.0
5,age_cat_55_64,240.0
6,age_cat_65_74,250.0
7,age_cat_75_more,235.0
8,sex_female,910.0
9,sex_male,855.0


In [11]:
summary_table_cases_otmedia_incdt

,variable,count
0,antibacterial_brit,1360.0
1,otmedia_ab_date,110.0
2,age_cat_15_24,155.0
3,age_cat_25_34,175.0
4,age_cat_35_44,145.0
5,age_cat_45_54,145.0
6,age_cat_55_64,145.0
7,age_cat_65_74,145.0
8,age_cat_75_more,170.0
9,sex_female,580.0


In [12]:
#merge hosped dfs
dfs_tab_cases = [summary_table_cases_lrti_incdt, summary_table_cases_lrti_prevl,  
                summary_table_cases_urti_combined_incdt, summary_table_cases_urti_combined_prevl, 
                summary_table_cases_uti_incdt, summary_table_cases_uti_prevl, 
                summary_table_cases_sinusitis_incdt, summary_table_cases_sinusitis_prevl, 
                summary_table_cases_otmedia_incdt, summary_table_cases_otmedia_prevl, 
                summary_table_cases_ot_externa_incdt, summary_table_cases_ot_externa_prevl, 
                summary_table_cases_urti_incdt, summary_table_cases_urti_prevl, 
                summary_table_cases_cough_incdt, summary_table_cases_cough_prevl, 
                summary_table_cases_cough_cold_incdt, summary_table_cases_cough_cold_prevl, 
                summary_table_cases_throat_incdt, summary_table_cases_throat_prevl, 
                ]

#replace any column that contains ab_date with ab_binray
for i in dfs_tab_cases:
    i.loc[i['variable'].str.contains('_ab_date'), 'variable'] = 'ab_binary'

pd.concat(dfs_tab_cases, axis=1, ignore_index=True)

#concat dfs and assign column names
# df_concated_cases = 
# pd.concat([j.set_index('variable') for j in dfs_tab_cases],axis=1)#, join='outer')#.reset_index()
#df_concated_cases#.columns = ['variable', 
#                         'lrti_incdt', 'lrti_prevl', 
#                         'urti_combined_incdt', 'urti_combined_prevl',
#                         'uti_incdt', 'uti_prevl',
#                         'sinusitis_incdt', 'sinusitis_prevl',
#                         'otmedia_incdt', 'otmedia_prevl',
#                         'ot_externa_incdt', 'ot_externa_prevl',
#                         'urti_incdt', 'urti_prevl',
#                         'cough_incdt', 'cough_prevl',
#                         'cough_cold_incdt', 'cough_cold_prevl',
#                         'throat_incdt', 'throat_prevl'
#                         ]

# #custom sort factors of variables column 
# df_concated_cases['variable'] = pd.Categorical(df_concated_cases['variable'], ['age_cat_15_24', 'age_cat_25_34', 'age_cat_35_44', 'age_cat_45_54', 'age_cat_55_64', 'age_cat_65_74', 'age_cat_75_more',
#                                                                                 'sex_female', 'sex_male',
#                                                                                 'CCI_cat_high', 'CCI_cat_low', 'CCI_cat_medium', 'CCI_cat_very_high', 'CCI_cat_very_low',
#                                                                                 'flu_vaccine_no', 'flu_vaccine_yes',
#                                                                                 'bmi_cat_healthy_weight', 'bmi_cat_obese', 'bmi_cat_overweight', 'bmi_cat_underweight', 'bmi_cat_unknown',
#                                                                                 'region_east', 'region_east_midlands', 'region_london', 'region_north_east', 'region_north_west', 'region_south_east', 'region_south_west', 'region_west_midlands', 'region_yorkshire',
#                                                                                 'imd_affluent', 'imd_medium', 'imd_unaffluent', 'imd_unknown', 'imd_very_affluent', 'imd_very_unaffluent',
#                                                                                 'ethnicity_asian', 'ethnicity_other', 'ethnicity_unknown', 'ethnicity_white',
#                                                                                 'smoking_ex_smoker', 'smoking_never_smoked', 'smoking_smoker', 'smoking_unknown',
#                                                                                 'season_autumn', 'season_spring', 'season_summer', 'season_winter',
#                                                                                 'period_during_pandemic', 'period_post_2nd_lockdown', 'period_prepandemic',
#                                                                                 'ab_binary', 'antibacterial_brit'])
# df_final_tab_cases = df_concated_cases.sort_values("variable").reset_index(drop=True)
# # df_final_tab_cases

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,antibacterial_brit,19800.0,antibacterial_brit,2140.0,antibacterial_brit,79385.0,antibacterial_brit,8840.0,antibacterial_brit,19770.0,...,antibacterial_brit,125.0,antibacterial_brit,1290.0,antibacterial_brit,160.0,antibacterial_brit,20080.0,antibacterial_brit,2150.0
1,age_cat_15_24,2335.0,age_cat_15_24,275.0,age_cat_15_24,9035.0,age_cat_15_24,980.0,age_cat_15_24,2305.0,...,age_cat_15_24,20.0,age_cat_15_24,145.0,age_cat_15_24,20.0,age_cat_15_24,2280.0,age_cat_15_24,230.0
2,age_cat_25_34,2200.0,age_cat_25_34,265.0,age_cat_25_34,9255.0,age_cat_25_34,1060.0,age_cat_25_34,2210.0,...,age_cat_25_34,20.0,age_cat_25_34,155.0,age_cat_25_34,15.0,age_cat_25_34,2285.0,age_cat_25_34,250.0
3,age_cat_35_44,2305.0,age_cat_35_44,265.0,age_cat_35_44,9120.0,age_cat_35_44,1030.0,age_cat_35_44,2330.0,...,age_cat_35_44,SM,age_cat_35_44,165.0,age_cat_35_44,15.0,age_cat_35_44,2255.0,age_cat_35_44,275.0
4,age_cat_45_54,2275.0,age_cat_45_54,235.0,age_cat_45_54,9100.0,age_cat_45_54,1015.0,age_cat_45_54,2345.0,...,age_cat_45_54,20.0,age_cat_45_54,145.0,age_cat_45_54,15.0,age_cat_45_54,2240.0,age_cat_45_54,255.0
5,age_cat_55_64,2320.0,age_cat_55_64,240.0,age_cat_55_64,9170.0,age_cat_55_64,1060.0,age_cat_55_64,2215.0,...,age_cat_55_64,15.0,age_cat_55_64,135.0,age_cat_55_64,25.0,age_cat_55_64,2330.0,age_cat_55_64,270.0
6,age_cat_65_74,2360.0,age_cat_65_74,250.0,age_cat_65_74,9215.0,age_cat_65_74,1075.0,age_cat_65_74,2255.0,...,age_cat_65_74,15.0,age_cat_65_74,145.0,age_cat_65_74,15.0,age_cat_65_74,2255.0,age_cat_65_74,240.0
7,age_cat_75_more,2300.0,age_cat_75_more,235.0,age_cat_75_more,9190.0,age_cat_75_more,995.0,age_cat_75_more,2280.0,...,age_cat_75_more,10.0,age_cat_75_more,130.0,age_cat_75_more,20.0,age_cat_75_more,2280.0,age_cat_75_more,275.0
8,sex_female,8245.0,sex_female,910.0,sex_female,32455.0,sex_female,3680.0,sex_female,8180.0,...,sex_female,50.0,sex_female,540.0,sex_female,70.0,sex_female,8035.0,sex_female,905.0
9,sex_male,7850.0,sex_male,855.0,sex_male,31630.0,sex_male,3535.0,sex_male,7760.0,...,sex_male,60.0,sex_male,485.0,sex_male,60.0,sex_male,7885.0,sex_male,885.0


In [ ]:
df_final_tab_cases#.head()

NameError: name 'df_final_tab_cases' is not defined

## 1- process data to make a table of counts and rates, and save it

In [ ]:
data_hosped = data_process(df_final_tab, df_final_tab_cases, 'lrti_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'lrti_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_combined_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'uti_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'sinusitis_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'otmedia_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'ot_externa_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'urti_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'cough_cold_prevl')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_incdt')
data_hosped = data_process(df_final_tab, data_hosped, 'throat_prevl')

#save hosped count and rate table
data_hosped.to_csv(f'../output/aggregate_cases_table/cases_count_rate_table_all_infections.csv', index=False)
data_hosped

,variable,lrti_incdt,lrti_prevl,urti_combined_incdt,urti_combined_prevl,uti_incdt,uti_prevl,sinusitis_incdt,sinusitis_prevl,otmedia_incdt,...,ot_externa_incdt,ot_externa_prevl,urti_incdt,urti_prevl,cough_incdt,cough_prevl,cough_cold_incdt,cough_cold_prevl,throat_incdt,throat_prevl
0,age_cat_15_24,"2,335 (101.2)",275 (107.0),"9,035 (99.6)",980 (96.8),"2,305 (100.1)",245 (95.7),"2,260 (99.8)",255 (98.8),"2,325 (100.7)",...,"2,280 (97.4)",275 (104.8),"2,250 (99.4)",235 (91.8),"2,295 (101.2)",255 (100.8),"2,210 (96.5)",260 (102.2),"2,280 (101.2)",230 (92.6)
1,age_cat_25_34,"2,200 (96.2)",265 (104.1),"9,255 (100.3)","1,060 (103.3)","2,210 (96.7)",260 (100.8),"2,280 (99.8)",255 (100.8),"2,250 (97.6)",...,"2,240 (98.9)",275 (109.6),"2,315 (100.4)",265 (101.5),"2,340 (101.4)",265 (100.6),"2,315 (99.7)",280 (107.9),"2,285 (99.7)",250 (102.9)
2,age_cat_35_44,"2,305 (99.9)",265 (105.0),"9,120 (100.8)","1,030 (101.1)","2,330 (100.1)",225 (89.8),"2,300 (100.1)",240 (88.9),"2,335 (100.4)",...,"2,300 (100.4)",280 (110.7),"2,220 (98.4)",270 (105.3),"2,315 (102.6)",250 (96.5),"2,330 (101.9)",235 (93.1),"2,255 (100.4)",275 (109.8)
3,age_cat_45_54,"2,275 (99.0)",235 (90.0),"9,100 (99.5)","1,015 (101.2)","2,345 (101.1)",265 (101.5),"2,340 (102.2)",270 (105.7),"2,315 (99.4)",...,"2,210 (96.6)",270 (106.7),"2,330 (102.2)",255 (100.4),"2,190 (96.6)",255 (102.2),"2,340 (101.3)",250 (99.4),"2,240 (97.8)",255 (102.8)
4,age_cat_55_64,"2,320 (98.7)",240 (91.1),"9,170 (100.4)","1,060 (103.8)","2,215 (98.1)",250 (96.5),"2,350 (101.5)",260 (98.3),"2,315 (99.3)",...,"2,350 (102.5)",245 (97.8),"2,320 (102.1)",260 (101.0),"2,290 (100.3)",250 (100.0),"2,230 (96.9)",280 (107.3),"2,330 (102.3)",270 (106.9)
5,age_cat_65_74,"2,360 (102.4)",250 (93.3),"9,215 (100.3)","1,075 (104.8)","2,255 (97.3)",275 (107.4),"2,360 (102.5)",245 (97.8),"2,260 (99.0)",...,"2,285 (101.4)",270 (105.7),"2,305 (98.9)",295 (112.6),"2,335 (102.1)",275 (108.3),"2,315 (100.9)",270 (106.5),"2,255 (99.0)",240 (93.8)
6,age_cat_75_more,"2,300 (99.8)",235 (89.4),"9,190 (101.1)",995 (97.4),"2,280 (99.5)",275 (105.8),"2,195 (96.7)",245 (95.7),"2,320 (99.9)",...,"2,270 (97.8)",275 (105.8),"2,305 (103.5)",245 (96.5),"2,395 (103.5)",225 (90.4),"2,215 (98.1)",250 (96.2),"2,280 (99.4)",275 (106.6)
7,sex_female,"8,245 (99.9)",910 (99.6),"32,455 (99.9)","3,680 (100.8)","8,180 (99.9)",885 (96.1),"8,125 (99.3)",900 (97.0),"8,195 (99.0)",...,"8,115 (99.0)",965 (105.6),"8,175 (101.1)",930 (100.9),"8,150 (99.5)",925 (100.9),"8,100 (99.4)",920 (100.5),"8,035 (99.6)",905 (101.1)
8,sex_male,"7,850 (99.3)",855 (94.4),"31,630 (100.7)","3,535 (101.6)","7,760 (98.1)",900 (102.3),"7,965 (101.6)",875 (99.4),"7,925 (99.9)",...,"7,820 (99.6)",925 (106.1),"7,870 (100.3)",900 (102.4),"8,020 (102.9)",845 (98.1),"7,855 (99.3)",900 (102.7),"7,885 (100.2)",885 (102.8)
9,CCI_cat_high,"1,635 (96.6)",180 (96.0),"6,720 (99.5)",750 (101.1),"1,680 (98.8)",200 (102.0),"1,670 (100.4)",180 (97.0),"1,715 (102.7)",...,"1,665 (98.3)",195 (104.3),"1,695 (101.2)",185 (98.1),"1,635 (97.9)",200 (108.4),"1,705 (99.7)",180 (95.7),"1,685 (99.1)",185 (102.5)
